In [17]:
# Libraries:
!pip install pke-tool
!python -m nltk.downloader stopwords
!python -m nltk.downloader universal_tagset
!python -m spacy download en
import pke
import nltk
import string
import re 
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords


C:\Users\nghuy\anaconda3\lib\runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nghuy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\nghuy\anaconda3\lib\runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\nghuy\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


[!] As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the full
pipeline package name 'en_core_web_sm' instead.
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\nghuy\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nghuy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nghuy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nghuy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# Import data and intial variable
train_var  = pd.read_csv('training_variants')
train_text = pd.read_csv('training_text', sep='\|\|', engine='python')
test_var   = pd.read_csv('test_variants')
test_text  = pd.read_csv('test_text', sep='\|\|', engine='python')

# Drop the rows whose text is NA and correct index in df after dropping elements
fixed_train_var        = train_var.drop([1109, 1277, 1407, 1639, 2755])
fixed_train_var.index  = range(len(fixed_train_var))                
fixed_train_text       = train_text.drop([1109, 1277, 1407, 1639, 2755])
fixed_train_text.index = range(len(fixed_train_text))

fixed_test_var        = test_var.drop(1623)
fixed_test_var.index  = range(len(fixed_test_var))
fixed_test_text       = test_text.drop(1623)
fixed_test_text.index = range(len(fixed_test_text))

In [20]:
# Extract keywords using TFIDF
def keywords_TFIDF(input_str, ngram, best):
    
    stop_words = stopwords.words('english')    # Common stop words in English
    extractor  = pke.unsupervised.TfIdf()      # Initiate unserpvised TFIDF
    
    extractor.load_document(input_str,language='en_core_web_sm',normalization=True)
    extractor.candidate_selection(n=ngram, stoplist=list(string.punctuation)+stop_words)
    extractor.candidate_weighting()
    
    keywords      = extractor.get_n_best(n=best,stemming=True,redundancy_removal=True) # Get the best n keywords
    keywords_only = [i for i, j in keywords]                                           # Remove the score from the keywords
    singleton     = list()  # List of single words
    phrase        = list()  # List of phrases
    
    # Differentiate singletons and phrases from the best candidates
    for text in keywords_only:
        if ' ' in text: 
          phrase.append(text)
        else :
          singleton.append(text)
        
    # Use tag of speech to filter out undesired word-types
    singleton_POS    = nltk.pos_tag(singleton)
    clean_singleton  = [word for word, tag in singleton_POS if tag not in ('VBN', 'VBZ','VBD','RB')]
    
    # Sort and remove duplicated keywords (singular vs plurals)
    sorted_singleton = sorted(clean_singleton)
    sorted_phrase    = sorted(phrase)
    final_singleton  = list()
    final_phrase     = list()
    
    for i in range(0,len(sorted_singleton)-1):
        if sorted_singleton[i] not in sorted_singleton[i+1]:
            final_singleton.append(sorted_singleton[i])
            
    for i in range(0,len(sorted_phrase)-1):
        if sorted_phrase[i] not in sorted_phrase[i+1]:
            final_phrase.append(sorted_phrase[i])
            
    final_singleton.append(sorted_singleton[-1])
    final_phrase.append(sorted_phrase[-1])
    
    
    return final_singleton,final_phrase

In [26]:
"""""
# Run this code for to extract keywords for the ith document in the train data

index = ?
text = fixed_train_text.iloc[index][0]
keywords_TFIDF(text,3,100)

"""""

'""\n# Run this code for to extract keywords for the ith document in the train data\n\nindex = ?\ntext = fixed_train_text.iloc[index][0]\nkeywords_TFIDF(text,3,100)\n\n'